In [175]:
from imutils.video import  WebcamVideoStream
import cv2
from imutils.video import FPS
from datetime import timedelta
import numpy as np

In [176]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    #clip_duration = cap.stream.get(cv2.CAP_PROP_FRAME_COUNT) / cap.stream.get(cv2.CAP_PROP_FPS)
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [177]:
#cap = WebcamVideoStream('test.mp4').start()
cap =cv2.VideoCapture(r'C:\FFOutput\tadem_00\tamdem_00_vd\IMG_5457.h264.mp4')
output_path = 'C:/FFOutput/tadem_00/'


In [178]:
SAVING_FRAMES_PER_SECOND = 20

In [179]:
fps_cus = cap.get(cv2.CAP_PROP_FPS)
print(fps_cus)
saving_frames_per_second = min(fps_cus, SAVING_FRAMES_PER_SECOND)
print(saving_frames_per_second)
saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
#print(saving_frames_durations)
print("total frame ", cap.get(cv2.CAP_PROP_FRAME_COUNT))


30.0
20
total frame  419.0


In [180]:
fps = FPS().start()

count = 0
while True:

    ok ,input_frame = cap.read()
    
    if not ok:
        break
    
    frame_duration = count / fps_cus

    #input_frame = cv2.resize(input_frame,(512,512),interpolation=cv2.INTER_AREA)

    try:
        # get the earliest duration to save
        closest_duration = saving_frames_durations[0]
    except IndexError:
        # the list is empty, all duration frames were saved
        break
    if frame_duration >= closest_duration:
        # if closest duration is less than or equals the frame duration, 
        # then save the frame
        frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))      
        cv2.imwrite( f"{output_path}frame_6{frame_duration_formatted}.jpg", input_frame) 
        # drop the duration spot from the list, since this duration spot is already saved
        try:
            saving_frames_durations.pop(0)
        except IndexError:
            pass
    # increment the frame count
    count = count+1

    #cv2.imwrite(f'{output_path}{i}.jpg', input_frame)
    cv2.imshow("origin", input_frame)
    cv2.waitKey(1)
    
    if cv2.waitKey(1) & 0xff ==ord('q'):
        break
    fps.update()
fps.stop()
cap.release()
cv2.destroyAllWindows()

In [181]:
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

[INFO] elasped time: 18.96
[INFO] approx. FPS: 22.10
